In [ ]:
from pyecsca.sca.trace_set import InspectorTraceSet, PickleTraceSet, HDF5TraceSet
from pyecsca.ec.params import get_params
from pyecsca.ec.mult import LadderMultiplier
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod

In [ ]:
curve25519 = get_params("other", "Curve25519", "xz")
p = curve25519.curve.prime
n = curve25519.order
ladder = curve25519.curve.coordinate_model.formulas["ladd-1987-m-3"]
dbl = curve25519.curve.coordinate_model.formulas["dbl-1987-m-3"]
mult = LadderMultiplier(ladder, dbl)

In [ ]:
unprotected_100 = InspectorTraceSet.read("sca25519_unprotected_100.trs", scale=False)

In [ ]:
def fix_trace(trace):
    data = trace.meta["data"]
    print(data.hex())
    group_data = bytearray([data[0]])
    
    point_data = bytearray(data[1:33])
    point_data[-1] &= 0b01111111
    
    scalar_data = bytearray(data[33:65])
    # From the impl:
    # https://github.com/sca-secure-library-sca25519/sca25519/blob/main/STM32F407-unprotected/crypto/scalarmult/scalarmult_25519.c#L125
    scalar_data[0] &= 248
    scalar_data[31] &= 127
    scalar_data[31] |= 64
    
    result_data = bytearray(data[65:])
    result_data[-1] &= 0b01111111
    
    point_x = int.from_bytes(point_data, byteorder="little")
    result_x = int.from_bytes(result_data, byteorder="little")
    scalar = int.from_bytes(scalar_data, byteorder="little")

    point = Point(curve25519.curve.coordinate_model, X=Mod(point_x, p), Z=Mod(1, p))
    result = Point(curve25519.curve.coordinate_model, X=Mod(result_x, p), Z=Mod(1, p))
    mult.init(curve25519, point)
    r = mult.multiply(scalar)
    mine = r.X / r.Z
    their = result.X / result.Z
    print(hex(int(mine)))
    print(hex(int(their)))
    print(mine == their)

for trace in unprotected_100[:1]:
    fix_trace(trace)